In [1]:
from pathlib import Path

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torchvision.datasets as datasets

from torchvision import transforms
from PIL import Image

In [21]:
# 數據預處理
transform = transforms.Compose([
    transforms.Resize((640, 640)),  # 調整大小為 32x32
    transforms.Grayscale(num_output_channels=1),  # 轉為單通道灰度圖像
    transforms.ToTensor(),  # 轉為張量
    transforms.Normalize((0.5,), (0.5,))  # 歸一化
])

train_dataset = datasets.ImageFolder(root='./dataset/train_images', transform=transform) # datasets.ImageFolder 會將資料夾中的圖片依照資料夾名稱分類
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=False)  #每次訓練數量 = Data set size(*0.8) / Batch size = 800/20 = 40

In [22]:
toPIL = transforms.ToPILImage()
a = 0

for i,_ in train_loader:
    toPIL(i[0][0]).save('./pic.jpg')
    print(_)



tensor([0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 1, 1, 1, 1])
tensor([1, 1, 1, 1, 1, 1, 1, 1])
tensor([1, 1, 1, 1, 1, 1, 1, 1])
tensor([2, 2, 2, 2, 2, 2, 2, 2])
tensor([2, 2, 2, 2, 2, 2, 2, 2])
tensor([2, 2, 2, 2, 3, 3, 3, 3])
tensor([3, 3, 3, 3, 3, 3, 3, 3])
tensor([3, 3, 3, 3, 3, 3, 3, 3])
tensor([4, 4, 4, 4, 4, 4, 4, 4])
tensor([4, 4, 4, 4, 4, 4, 4, 4])
tensor([4, 4, 4, 4, 5, 5, 5, 5])
tensor([5, 5, 5, 5, 5, 5, 5, 5])
tensor([5, 5, 5, 5, 5, 5, 5, 5])
tensor([6, 6, 6, 6, 6, 6, 6, 6])
tensor([6, 6, 6, 6, 6, 6, 6, 6])
tensor([6, 6, 6, 6, 7, 7, 7, 7])
tensor([7, 7, 7, 7, 7, 7, 7, 7])
tensor([7, 7, 7, 7, 7, 7, 7, 7])
tensor([8, 8, 8, 8, 8, 8, 8, 8])
tensor([8, 8, 8, 8, 8, 8, 8, 8])
tensor([8, 8, 8, 8, 9, 9, 9, 9])
tensor([9, 9, 9, 9, 9, 9, 9, 9])
tensor([9, 9, 9, 9, 9, 9, 9, 9])
tensor([10, 10, 10, 10, 10, 10, 10, 10])
tensor([10, 10, 10, 10, 10, 10, 10, 10])
tensor([10, 10, 10, 10, 11, 11, 11, 11])
tensor([11, 11, 11, 11, 11, 11, 11, 11])
tensor([11,